In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

title = []
episodes = []
status = []
aired = []
genres = []
favorites = []
members = []
popularity = []
rank = []
score = []
studios = []
synopsis = []
type_ = []
theme = []
demographic = []
duration = []
premiered = []

n=0

for limit in range(18900, 19000, 50):
    url = f"https://myanimelist.net/topanime.php?limit={limit}"
    page = requests.get(url)
    doc = BeautifulSoup(page.content, 'html.parser')
    
    for item in doc.find_all(class_='ranking-list'):
        n+=1
        print(n, end=' ') 
        
        title.append(item.find(class_='hoverinfo_trigger fl-l fs14 fw-b anime_ranking_h3').text)
        
        link = item.find(class_='hoverinfo_trigger fl-l fs14 fw-b anime_ranking_h3').a.get('href')
        item_page = requests.get(link)
        item_doc = BeautifulSoup(item_page.content, 'html.parser')
        
        time.sleep(2)

        try:
            synopsis.append(item_doc.find(itemprop="description").text)
        except:
            synopsis.append(None)
            
        all_info = item_doc.find(class_='borderClass').find_all(class_='spaceit_pad')
    
        for info in all_info:
            cur_info = info.text.strip().split(':')
            if cur_info[0] == 'Premiered':
                premiered.append(cur_info[1].strip())
            elif cur_info[0] == 'Episodes':
                episodes.append(cur_info[1].strip())
            elif cur_info[0] == 'Status':
                status.append(cur_info[1].strip())
            elif cur_info[0] == 'Aired':
                aired.append(cur_info[1].strip())
            elif cur_info[0] == 'Genres' or cur_info[0] == 'Genre':
                cur_genre = ''
                genre_list = cur_info[1].split(' ')
                for i in genre_list:
                    if i != '':
                        if i[-1] == ',':
                            cur_genre += i
                cur_genre += genre_list[-1]

                if cur_genre == '':
                    genres.append('Unknown')
                else:
                    append_genres = ''
                    for i in cur_genre.split(','):
                        append_genres += i.strip()[len(i)//2:] + ','
                    genres.append(append_genres)
            elif cur_info[0] == 'Studios':
                studios.append(cur_info[1].strip())
            elif cur_info[0] == 'Score':
                score.append(cur_info[1].strip().split(' ')[0][:-1])
            elif cur_info[0] == 'Ranked':
                rank.append(cur_info[1].strip().split(' ')[0][:-1].split('\n')[0][:-1])
            elif cur_info[0] == 'Popularity':
                popularity.append(cur_info[1].strip())
            elif cur_info[0] == 'Members':
                members.append(cur_info[1].strip())
            elif cur_info[0] == 'Favorites':
                favorites.append(cur_info[1].strip())
            elif cur_info[0] == 'Theme':
                theme.append(cur_info[1].strip()[len(cur_info[1].strip())//2:])
            elif cur_info[0] == 'Demographic':
                demographic.append(cur_info[1].strip()[len(cur_info[1].strip())//2:])
            elif cur_info[0] == 'Duration':
                duration.append(cur_info[1].strip())
            elif cur_info[0] == 'Type':
                type_.append(cur_info[1].strip())
            
        

        if len(genres) != len(members):
            genres.append('Unknown')
        if len(demographic) != len(members):
            demographic.append('Unknown')
        if len(theme) != len(members):
            theme.append('Unknown')
        if len(premiered) != len(members):
            premiered.append('Unknown')

In [ ]:
mal_anime_df = pd.DataFrame({'title': title, 'episodes': episodes,  'status': status, 'theme': theme, 
                             'demographic': demographic, 'genres': genres, 'type': type_,
                             'favorites': favorites, 'popularity': popularity, 'rank': rank, 'score': score, 
                             'members': members, 'synopsis': synopsis, 'aired': aired, 'duration': duration, 
                             'premiered': premiered,'studios': studios})

mal_anime_df.to_csv('18900-19000anime_mal.csv')

In [ ]:
file_list = []
df_list = []
for i in range(0,25000,1000):
    file = f"{i}-{i+1000}anime_mal.csv"
    df_list.append(pd.read_csv(file))

anime_df = pd.concat(df_list).drop(['Unnamed: 0'], axis=1)
anime_df = anime_df.reset_index()
anime_df.drop('index',axis=1).to_csv('mal_anime.csv')

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

score_10 = []
score_9 = []
score_8 = []
score_7 = []
score_6 = []
score_5 = []
score_4 = []
score_3 = []
score_2 = []
score_1 = []
watching = []
completed = []
on_hold = []
dropped = []
plan_to_watch = []
n=0

for limit in range(0, 10000, 50):
    url = f"https://myanimelist.net/topanime.php?limit={limit}"
    page = requests.get(url)
    doc = BeautifulSoup(page.content, 'html.parser')
    
    for item in doc.find_all(class_='ranking-list'):
        n+=1
        print(n, end=' ') 
        if n%100 == 0:
            time.sleep(60)
        
        link = item.find(class_='hoverinfo_trigger fl-l fs14 fw-b anime_ranking_h3').a.get('href')
        item_page = requests.get(link+'/stats')
        item_doc = BeautifulSoup(item_page.content, 'html.parser')
        all_stat = item_doc.find_all(class_='spaceit_pad')
                
        for info in all_stat:
            cur_info = info.text.strip().split(':')
            if cur_info[0] == 'Watching':
                watching.append(cur_info[1].strip())
            elif cur_info[0] == 'Completed':
                completed.append(cur_info[1].strip())
            elif cur_info[0] == 'On-Hold':
                on_hold.append(cur_info[1].strip())
            elif cur_info[0] == 'Dropped':
                dropped.append(cur_info[1].strip())
            elif cur_info[0] == 'Plan to Watch':
                plan_to_watch.append(cur_info[1].strip())
                
        score_ = 10
        
        all_scores = item_doc.find(class_='score-stats')
            
        all_scores_ind = all_scores.find_all('div', {'class':'spaceit_pad'})
       
            
        for score in all_scores_ind:
            if score_ == 10:
                score_10.append(score.small.text.split(' ')[0][1:])
            elif score_ == 9:
                score_9.append(score.small.text.split(' ')[0][1:])
            elif score_ == 8:
                score_8.append(score.small.text.split(' ')[0][1:])
            elif score_ == 7:
                score_7.append(score.small.text.split(' ')[0][1:])
            elif score_ == 6:
                score_6.append(score.small.text.split(' ')[0][1:])
            elif score_ == 5:
                score_5.append(score.small.text.split(' ')[0][1:])
            elif score_ == 4:
                score_4.append(score.small.text.split(' ')[0][1:])
            elif score_ == 3:
                score_3.append(score.small.text.split(' ')[0][1:])
            elif score_ == 2:
                score_2.append(score.small.text.split(' ')[0][1:])
            elif score_ == 1:
                score_1.append(score.small.text.split(' ')[0][1:])

            score_ -= 1

In [ ]:
mal_anime_stat = pd.DataFrame({'watching': watching, 'completed': completed, 'on_hold': on_hold, 'dropped': dropped,
                               'plan_to_watch': plan_to_watch,
                               'score10': score_10, 'score9': score_9, 'score8': score_8, 'score7': score_7, 
                               'score6': score_6, 'score5': score_5, 'score4': score_4, 'score3': score_3,
                               'score2': score_2, 'score2': score_1})

mal_anime_stat.to_csv('0-10000animestats_mal.csv')